In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
import os
# aggiungi la cartella Scripts al path di ricerca moduli
scripts_path = os.path.abspath(os.path.join('..', 'Scripts'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

from funzioni import trimestri_stats, prezzo_trimestri_stats

In [2]:
# -------------------------------
#  Lettura e preparazione dati
# -------------------------------
iphone14 = pd.read_csv("/Users/federicogamberini/VS Code/when-to-buy-iphone/Dataset/iPhone14.csv") 
iphone15 = pd.read_csv("/Users/federicogamberini/VS Code/when-to-buy-iphone/Dataset/iPhone15.csv") 
iphone16 = pd.read_csv("/Users/federicogamberini/VS Code/when-to-buy-iphone/Dataset/iPhone16.csv")

combined = pd.concat([iphone14, iphone15, iphone16], ignore_index=True)
iphone14_anno1 = iphone14[iphone14['Giorni_dal_lancio']<=365]
iphone15_anno1 = iphone15[iphone15['Giorni_dal_lancio']<=365]

stats_14 = trimestri_stats(iphone14, "iPhone14")
stats_15 = trimestri_stats(iphone15, "iPhone15")
stats_16 = trimestri_stats(iphone16, "iPhone16")
df_statistiche = pd.concat([stats_14, stats_15, stats_16], ignore_index=True)

# Ordine trimestri e aggiunta colonna Trimestre "Q..."
ordine_trimestri = ['T1', 'T2', 'T3', 'T4']
df_statistiche['Trimestre'] = pd.Categorical(df_statistiche['Trimestre'], categories=ordine_trimestri, ordered=True)
# Calcolo giorno medio per trimestre per plotting 
trimestre_to_giorni = {'T1': (0, 90),'T2': (91, 180),'T3': (181, 270),'T4': (271, 365)}
df_statistiche['Giorni_dal_lancio'] = df_statistiche['Trimestre'].map(
    lambda q: sum(trimestre_to_giorni[q]) // 2 if pd.notna(q) else None)

trimestri_validi = ordine_trimestri  #trimestri_validi = df_statistiche['Trimestre'].dropna().unique()

In [3]:
# -------------------------------
#           Plotting    
# -------------------------------

# Colori
palette = {
    'iPhone14': '#FFA500',  # orange → leggermente più saturo
    'iPhone15': '#FF4C4C',  # rosso acceso ma non neon
    'iPhone16': '#4CA3FF',  # blu chiaro brillante
}
trimestre_color = {
    'T1': '#CCCCCC',  # grigio chiaro
    'T2': '#90EE90',  # verde chiaro
    'T3': '#FFD580',  # arancio chiaro
    'T4': '#DA70D6',  # viola orchid
}

fig = make_subplots(rows=2, cols=2,row_heights=[1, 1.3],  # proporzioni relative
    specs=[[{}, {}], [{"colspan": 2}, None]],  # l'ultima riga ha un grafico che occupa entrambe le colonne
    subplot_titles=("Prezzo nel Primo Anno","Prezzo Medio per Trimestre","Prezzi Giornalieri + Prezzi Medi Trimestrali"))

#Primo plot: Prezzo nel Primo Anno
fig.add_trace(go.Scatter(x=iphone14_anno1['Giorni_dal_lancio'],y=iphone14_anno1['Prezzo'],mode='lines',name='iPhone 14 giornaliero',legendgroup='Giornaliero',legendgrouptitle_text='Giornaliero',line=dict(width=1, color='#FFA500')),row=1, col=1)
fig.add_trace(go.Scatter(x=iphone15_anno1['Giorni_dal_lancio'],y=iphone15_anno1['Prezzo'],mode='lines',name='iPhone 15 giornaliero',legendgroup='Giornaliero',line=dict(width=1, color='#FF4C4C')),row=1, col=1)
fig.add_trace(go.Scatter(x=iphone16['Giorni_dal_lancio'],y=iphone16['Prezzo'],mode='lines',name='iPhone 16 giornaliero',legendgroup='Giornaliero',line=dict(width=1,color='#4CA3FF')),row=1, col=1)

#Secondo plot: Prezzo Medio per Trimestre
for modello in df_statistiche['Modello'].unique():
    df_mod = df_statistiche[(df_statistiche['Modello'] == modello) &(df_statistiche['Trimestre'].isin(trimestri_validi))]
    fig.add_trace(go.Scatter(x=df_mod['Trimestre'],y=df_mod['mean'],mode='lines+markers',name=f"{modello} Trimestre, medio",legendgroup='Trimestre, medio', legendgrouptitle_text='Trimestre, medio',line=dict(color=palette[modello])),row=1, col=2)
    
#Terzo plot: Prezzi Giornalieri + Prezzi Medi Trimestrali
for trimestre, (start, end) in trimestre_to_giorni.items():
    fig.add_vrect(x0=start, x1=end,fillcolor=trimestre_color[trimestre],opacity=0.2,layer="below",row=2, col=1)
    fig.add_annotation(x=(start + end) / 2,y=df_statistiche['mean'].max() * 1.02,text=trimestre,showarrow=False,font=dict(size=10, color='white'),row=2, col=1)
for modello in combined['Modello'].unique():
    df_comb = combined[(combined['Modello'] == modello) & (combined['Giorni_dal_lancio'] <= 365)]
    fig.add_trace(go.Scatter(x=df_comb['Giorni_dal_lancio'],y=df_comb['Prezzo'],mode='lines',line=dict(width=1, color=palette[modello]),opacity=0.5,name=f"{modello} giornaliero",legendgroup='Giornaliero',legendgrouptitle_text='Giornaliero'),row=2, col=1)
for modello in df_statistiche['Modello'].unique():
    df_mod = df_statistiche[(df_statistiche['Modello'] == modello) & (df_statistiche['Trimestre'].isin(trimestri_validi)) & (df_statistiche['Giorni_dal_lancio'] <= 365)]
    fig.add_trace(go.Scatter(x=df_mod['Giorni_dal_lancio'],y=df_mod['mean'],mode='lines+markers',line=dict(width=2, color=palette[modello]),name=f"{modello} Trimestre, medio", legendgroup='Trimestre, medio', legendgrouptitle_text='Trimestre, medio',),row=2, col=1)

fig.update_xaxes(title_text="Giorni dal Lancio", row=2, col=1)
fig.update_xaxes(title_text="Giorni dal Lancio", row=1, col=1)
fig.update_yaxes(title_text="Prezzo (€)", row=1, col=1)
fig.update_yaxes(title_text="Prezzo Medio (€)", row=1, col=2)
fig.update_yaxes(title_text="Prezzo (€)", row=2, col=1)

fig.update_layout(height=900,width=1200,legend=dict(title="Modello"),title_text="Confronto Prezzi iPhone", template='plotly_dark')

fig.show()

In [4]:
# -------------------------------
#    Statistiche Riassuntive
# -------------------------------
resoconto_14 = iphone14.groupby(['Trimestre'], observed=True).apply(prezzo_trimestri_stats, include_groups=False).reset_index()
resoconto_15 = iphone15.groupby(['Trimestre'], observed=True).apply(prezzo_trimestri_stats, include_groups=False).reset_index()
resoconto_16 = iphone16.groupby(['Trimestre'], observed=True).apply(prezzo_trimestri_stats, include_groups=False).reset_index()

In [5]:
print(f"iPhone 14 - Resoconto Prezzi per Trimestre:")
resoconto_14

iPhone 14 - Resoconto Prezzi per Trimestre:


,Trimestre,Prezzo Iniziale,Prezzo Finale,Variazione Totale (€),Variazione Totale (%),Prezzo Medio Trimestre,Oscillazione Massima (€),Volatilità (std dev)
0,T1,1029.0,999.0,-30.0,-2.92,1007.46,130.0,35.053073
1,T2,949.0,879.0,-70.0,-7.38,905.56,70.0,21.102272
2,T3,899.0,829.0,-70.0,-7.79,860.89,100.0,39.317047
3,T4,829.0,799.0,-30.0,-3.62,819.53,30.0,14.018864
4,T5,799.0,749.0,-50.0,-6.26,781.92,50.0,22.729533
5,T6,749.0,699.0,-50.0,-6.68,729.33,50.0,19.856225
6,T7,699.0,699.0,0.0,0.00,692.16,50.0,17.275192
7,T8,699.0,699.0,0.0,0.00,699.00,0.0,0.000000


In [6]:
print(f"iPhone 15 - Resoconto Prezzi per Trimestre:")
resoconto_15

iPhone 15 - Resoconto Prezzi per Trimestre:


,Trimestre,Prezzo Iniziale,Prezzo Finale,Variazione Totale (€),Variazione Totale (%),Prezzo Medio Trimestre,Oscillazione Massima (€),Volatilità (std dev)
0,T1,979.0,839.0,-140.0,-14.30,947.78,140.00,40.608511
1,T2,879.0,799.0,-80.0,-9.10,843.50,180.00,36.163236
2,T3,799.0,779.0,-20.0,-2.50,780.64,50.00,16.290895
3,T4,779.0,759.0,-20.0,-2.57,769.00,70.00,11.485421
4,T5,759.0,699.0,-60.0,-7.91,735.05,107.01,29.535575
5,T6,699.0,699.0,0.0,0.00,729.96,50.00,15.457845
6,T7,699.0,669.0,-30.0,-4.29,691.73,84.00,22.566655
7,T8,669.0,659.0,-10.0,-1.49,664.88,40.00,10.036697


In [7]:
print(f"iPhone 16 - Resoconto Prezzi per Trimestre:")
resoconto_16

iPhone 16 - Resoconto Prezzi per Trimestre:


,Trimestre,Prezzo Iniziale,Prezzo Finale,Variazione Totale (€),Variazione Totale (%),Prezzo Medio Trimestre,Oscillazione Massima (€),Volatilità (std dev)
0,T1,979.00,879.0,-100.00,-10.21,945.91,140.00,45.062622
1,T2,831.99,849.0,17.01,2.04,854.87,97.99,22.011522
2,T3,849.00,769.0,-80.00,-9.42,784.53,94.00,20.243573
3,T4,769.00,779.0,10.00,1.30,774.79,10.00,5.072573
